In [ ]:
# ===== Модули Обновлено =====

from grid.model.navigation.objectinspection import ObjectInspect
from grid.model.navigation.visualservoing import VisualServoing
from grid.model.perception.detection.rt_detr import RT_DETR

import airgen
import numpy as np
import rerun as rr
import time
from math import cos, sin, pi

# Инициализация моделей
nav_inspect = ObjectInspect()
nav_visualservoing = VisualServoing()
detection_rt_detr = RT_DETR()
detection_rt_detr_0 = RT_DETR()

In [ ]:
# ===== Подключение к дрону и взлёт =====

client = airgen.MultirotorClient()
client.confirmConnection()
client.enableApiControl(True)
client.armDisarm(True)
client.takeoffAsync().join()

Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)



In [ ]:
# ===== Создание клиента сцены =====
scene_client = airgen.VehicleClient()
scene_objects = scene_client.simListSceneObjects()
cameras = [obj for obj in scene_objects if 'camera' in obj.lower()]

print("Available cameras:")
for camera in cameras:
    print(camera)

Available cameras:
PlayerCameraManager_2147482445
CameraManagerActor
ExternalCamera
BP_PIPCamera_C_2147482405
BP_PIPCamera_C_2147482393
BP_PIPCamera_C_2147482381
BP_PIPCamera_C_2147482369
BP_PIPCamera_C_2147482357


In [ ]:
# ===== Получение данных LiDAR + визуализация =====
lidar_data = client.getLidarData()
points = np.array(lidar_data.point_cloud, dtype=np.float32).reshape(-1, 3)
rr.log('lidar/points', rr.Points3D(points))

In [ ]:
# ===== Получение и логирование изображения с камеры =====
image_data = client.getImages("front_center", [airgen.ImageType.Scene])[0]
rgb_image, _ = image_data  # Получаем изображение и позу
rr.log("drone/camera", rr.Image(rgb_image))

In [ ]:
# ===== Получение и логирование GPS =====
gps_data = client.getGpsData()
geo = gps_data.gnss.geo_point
gps_position = np.array([[geo.latitude, geo.longitude, geo.altitude]], dtype=np.float32)
rr.log("drone/gps", rr.Points3D(gps_position))